In [6]:
import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
df = pd.read_pickle('Datasets/combined_data.pk1')
print(df.head())

MemoryError: 

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

print(df.head())

In [ ]:
print(df.isna().sum())

In [ ]:
print(df.dtypes)

In [ ]:
df.shape

In [ ]:
df = df[df['longitude'] >= -20]

In [ ]:
plt.scatter(df['longitude'], df['latitude'], c=df['altitude'], cmap='viridis', alpha=0.5)
plt.title('Flight Trajectories')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.colorbar(label='Altitude')
plt.show()

In [ ]:
grouped_flights = df.groupby('icao24')
flight_dfs = {icao24: group for icao24, group in grouped_flights}

In [ ]:
flight_dfs

In [ ]:
# Defining coordinates for Barcelona (BCN) and Amsterdam (AMS) airports with a tolerance range
barcelona_coords = (41.2974, 2.0833)  # BCN Airport
amsterdam_coords = (52.3105, 4.7683)  # AMS Airport

# Tolerance range in degrees, can be adjusted
tolerance = 0.5  # Approx. 55 km

# Define ranges for latitude and longitude for both airports
barcelona_lat_range = (barcelona_coords[0] - tolerance, barcelona_coords[0] + tolerance)
barcelona_lon_range = (barcelona_coords[1] - tolerance, barcelona_coords[1] + tolerance)

amsterdam_lat_range = (amsterdam_coords[0] - tolerance, amsterdam_coords[0] + tolerance)
amsterdam_lon_range = (amsterdam_coords[1] - tolerance, amsterdam_coords[1] + tolerance)

# Function to check if a point is within a given range
def in_range(lat, lon, lat_range, lon_range):
    return lat_range[0] <= lat <= lat_range[1] and lon_range[0] <= lon <= lon_range[1]

# Filter flights
barcelona_amsterdam_flights = {}

for icao24, flight_df in flight_dfs.items():
    # Sort by timestamp to ensure the order of points
    flight_df_sorted = flight_df.sort_values(by='timestamp')
    # Get the first and last position of the flight
    first_position = flight_df_sorted.iloc[0]
    last_position = flight_df_sorted.iloc[-1]

    # Check if the flight starts in Barcelona and ends in Amsterdam
    if in_range(first_position['latitude'], first_position['longitude'], barcelona_lat_range, barcelona_lon_range) and \
       in_range(last_position['latitude'], last_position['longitude'], amsterdam_lat_range, amsterdam_lon_range):
        barcelona_amsterdam_flights[icao24] = flight_df

# Number of flights identified
len(barcelona_amsterdam_flights)

In [ ]:
with open('barcelona_amsterdam_flights.pkl', 'wb') as file:
    pickle.dump(barcelona_amsterdam_flights, file)